In [19]:
# Lets import all required objects
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [20]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [21]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [22]:
cdk = Website("https://www.cdkglobal.com/")
print(cdk.get_contents())
print ("Links:")
print (cdk.links)


Webpage Title:
CDK Global: Homepage
Webpage Contents:
CDK Global
CDK Global company logo
Who We Serve
Auto Dealerships
Original Equipment Manufacturers
Independent Software Vendors
Heavy Truck
Friction Points Study 2025
In our latest Friction Points study, we uncover where the industry has improved and what still creates friction for customers.
Explore More Details
Dealership Xperience Platform
Foundations Suite
Strengthen workflows in every department with one complete solution
Modern Retail Suite
Surpass consumer expectations by making the buying journey easier
Fixed Operations Suite
Build loyalty and efficiency to maximize your Parts and Service income
Vehicle Inventory Suite
Price more profitably and elevate merchandising with the power of AI
Intelligence Suite
Fine-tune performance with advanced analytics, AI and machine learning
What We Offer
Inventory
Digital Retail
Customer Relationship Management
Finance & Insurance
Dealer Management System
Fixed Operations
Network & Communica

In [23]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [24]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [25]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Include all the product offerings of the company. Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [26]:
print(get_links_user_prompt(cdk))

Here is the list of links on the website of https://www.cdkglobal.com/ - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Include all the product offerings of the company. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
/
/automotive
/oem-offerings
/independent-software-vendors
https://www.cdkglobalheavytruck.com
/insights/dealers-speed-sales-according-latest-cdk-friction-points-study
/dealership-xperience-platform
/cdk-foundations-suite
/cdk-modern-retail-suite
/cdk-fixed-operations-suite
/cdk-vehicle-inventory-suite
/cdk-intelligence-suite
/cdk-vehicle-inventory-suite
/automotive-digital-retailing
/sales-inventory/vehicle-sales/automotive-crm-software
/fi
/dms
/fixed-ops
/it-solutions
/fueling-dealerships-harnessing-data-and-intelligence
/cdk-unify
/about
https://careers.cdkglobal.com
/insights
/training
/media-center
/cdk-trust-center
/customer-stories
/s

In [27]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [28]:
get_links("https://www.cdkglobal.com/")

{'links': [{'type': 'about page', 'url': 'https://www.cdkglobal.com/about'},
  {'type': 'careers page', 'url': 'https://careers.cdkglobal.com'},
  {'type': 'dealership xperience platform',
   'url': 'https://www.cdkglobal.com/dealership-xperience-platform'},
  {'type': 'cdk foundations suite',
   'url': 'https://www.cdkglobal.com/cdk-foundations-suite'},
  {'type': 'cdk modern retail suite',
   'url': 'https://www.cdkglobal.com/cdk-modern-retail-suite'},
  {'type': 'cdk fixed operations suite',
   'url': 'https://www.cdkglobal.com/cdk-fixed-operations-suite'},
  {'type': 'cdk vehicle inventory suite',
   'url': 'https://www.cdkglobal.com/cdk-vehicle-inventory-suite'},
  {'type': 'cdk intelligence suite',
   'url': 'https://www.cdkglobal.com/cdk-intelligence-suite'},
  {'type': 'automotive digital retailing',
   'url': 'https://www.cdkglobal.com/automotive-digital-retailing'},
  {'type': 'automotive crm software',
   'url': 'https://www.cdkglobal.com/sales-inventory/vehicle-sales/automo

In [29]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [30]:
# print(get_all_details("https://www.cdkglobal.com/"))
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [31]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [32]:
get_brochure_user_prompt("CDK Global", "https://www.cdkglobal.com/")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.cdkglobal.com/about'}, {'type': 'careers page', 'url': 'https://careers.cdkglobal.com'}, {'type': 'automotive offering', 'url': 'https://www.cdkglobal.com/automotive'}, {'type': 'OEM offerings', 'url': 'https://www.cdkglobal.com/oem-offerings'}, {'type': 'independent software vendors', 'url': 'https://www.cdkglobal.com/independent-software-vendors'}, {'type': 'dealership experience platform', 'url': 'https://www.cdkglobal.com/dealership-xperience-platform'}, {'type': 'CDK foundations suite', 'url': 'https://www.cdkglobal.com/cdk-foundations-suite'}, {'type': 'CDK modern retail suite', 'url': 'https://www.cdkglobal.com/cdk-modern-retail-suite'}, {'type': 'CDK fixed operations suite', 'url': 'https://www.cdkglobal.com/cdk-fixed-operations-suite'}, {'type': 'CDK vehicle inventory suite', 'url': 'https://www.cdkglobal.com/cdk-vehicle-inventory-suite'}, {'type': 'CDK intelligence suite', 'url': 'https://www.cdkglobal.com/cdk

"You are looking at a company called: CDK Global\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nCDK Global: Homepage\nWebpage Contents:\nCDK Global\nCDK Global company logo\nWho We Serve\nAuto Dealerships\nOriginal Equipment Manufacturers\nIndependent Software Vendors\nHeavy Truck\nFriction Points Study 2025\nIn our latest Friction Points study, we uncover where the industry has improved and what still creates friction for customers.\nExplore More Details\nDealership Xperience Platform\nFoundations Suite\nStrengthen workflows in every department with one complete solution\nModern Retail Suite\nSurpass consumer expectations by making the buying journey easier\nFixed Operations Suite\nBuild loyalty and efficiency to maximize your Parts and Service income\nVehicle Inventory Suite\nPrice more profitably and elevate merchandising with the power of AI\nIntelligence

In [33]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [34]:
create_brochure("CDK Global", "https://www.cdkglobal.com/")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.cdkglobal.com/about'}, {'type': 'careers page', 'url': 'https://careers.cdkglobal.com'}, {'type': 'automotive offerings', 'url': 'https://www.cdkglobal.com/automotive'}, {'type': 'oem offerings', 'url': 'https://www.cdkglobal.com/oem-offerings'}, {'type': 'independent software vendors', 'url': 'https://www.cdkglobal.com/independent-software-vendors'}, {'type': 'CDK Heavy Truck', 'url': 'https://www.cdkglobalheavytruck.com'}, {'type': 'dealership xperience platform', 'url': 'https://www.cdkglobal.com/dealership-xperience-platform'}, {'type': 'CDK Foundations Suite', 'url': 'https://www.cdkglobal.com/cdk-foundations-suite'}, {'type': 'CDK Modern Retail Suite', 'url': 'https://www.cdkglobal.com/cdk-modern-retail-suite'}, {'type': 'CDK Fixed Operations Suite', 'url': 'https://www.cdkglobal.com/cdk-fixed-operations-suite'}, {'type': 'CDK Vehicle Inventory Suite', 'url': 'https://www.cdkglobal.com/cdk-vehicle-inventory-suite'

# CDK Global Brochure

## Who We Are
For over 50 years, **CDK Global** has been at the forefront of providing innovative tools and technology tailored specifically for the automotive retail industry. Our mission is to empower dealers by connecting them with world-class dealership software solutions, driving efficiency, and enhancing customer experiences.

## What We Offer
CDK Global specializes in a wide range of solutions designed to streamline dealership operations and improve customer interactions:

- **Dealership Xperience Platform**: A comprehensive solution that optimizes the selling and servicing of vehicles.
- **Digital Retail**: Create a seamless customer experience both online and in-store.
- **Customer Relationship Management (CRM)**: Focus on customer-centric growth strategies.
- **Finance & Insurance (F&I)**: Maximize profitability while enhancing customer satisfaction.
- **Dealer Management System (DMS)**: A powerful system that integrates all dealership functions.
- **Fixed Operations**: Improve efficiency in service and repairs.
- **Network & Communication Services**: Ensure your IT infrastructure supports your business needs.
- **Intelligence**: Leverage data analytics and insights to enhance performance.

Discover more about our offerings and how we can help your dealership reach its full potential on our [website](https://www.cdkglobal.com).

## Who We Serve
CDK Global serves a diverse clientele in the automotive industry, including:
- Auto Dealerships
- Original Equipment Manufacturers (OEMs)
- Independent Software Vendors
- Heavy Truck Industries

We cater to nearly **15,000 dealer locations**, providing tailored solutions that address the specific challenges faced in automotive retail.

## Company Culture
At CDK Global, we foster a culture of innovation, inclusivity, and collaboration. Our team is dedicated to creating connections that facilitate effective automotive retail practices. We believe that our employees are our greatest asset, and we continuously seek to cultivate an environment where creativity thrives and ideas are encouraged.

## Careers at CDK
As a leader in our industry, we are always on the lookout for the **best and brightest talent** to join our team. We offer a range of career opportunities that promote growth and learning. Joining CDK Global means becoming part of a dynamic team that is committed to transforming the automotive retail landscape.

Explore our career opportunities and learn how you can be part of our mission on our [Careers](https://www.cdkglobal.com/careers) page.

## Contact Us
**Phone**: 833.620.2129

For more information about our products, insights, training, and customer support, please visit our [website](https://www.cdkglobal.com).

---

Empower your dealership with CDK Global—where automotive innovation meets exceptional service.

In [35]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [36]:
stream_brochure("CDK Global", "https://www.cdkglobal.com/")


Found links: {'links': [{'type': 'about page', 'url': 'https://www.cdkglobal.com/about'}, {'type': 'careers page', 'url': 'https://careers.cdkglobal.com'}, {'type': 'CDK Foundations Suite', 'url': 'https://www.cdkglobal.com/cdk-foundations-suite'}, {'type': 'CDK Modern Retail Suite', 'url': 'https://www.cdkglobal.com/cdk-modern-retail-suite'}, {'type': 'CDK Fixed Operations Suite', 'url': 'https://www.cdkglobal.com/cdk-fixed-operations-suite'}, {'type': 'CDK Vehicle Inventory Suite', 'url': 'https://www.cdkglobal.com/cdk-vehicle-inventory-suite'}, {'type': 'CDK Intelligence Suite', 'url': 'https://www.cdkglobal.com/cdk-intelligence-suite'}, {'type': 'Dealership Xperience Platform', 'url': 'https://www.cdkglobal.com/dealership-xperience-platform'}, {'type': 'Automotive Digital Retailing', 'url': 'https://www.cdkglobal.com/automotive-digital-retailing'}, {'type': 'Automotive CRM Software', 'url': 'https://www.cdkglobal.com/sales-inventory/vehicle-sales/automotive-crm-software'}, {'type':

# CDK Global Brochure

## Welcome to CDK Global

![CDK Global Logo](link-to-logo-image)

### Who We Serve
CDK Global is a trusted partner for nearly 15,000 dealer locations, including:
- **Auto Dealerships**
- **Original Equipment Manufacturers**
- **Independent Software Vendors**
- **Heavy Truck Dealers**

### Our Solutions
At CDK Global, we deliver industry-leading software solutions designed to enhance dealership performance across key operational areas. Our offerings include:

- **Dealership Xperience Platform**: Transform how you sell and service vehicles with an integrated, customer-focused approach.
- **Modern Retail Suite**: Ease the buying journey and exceed consumer expectations.
- **Fixed Operations Suite**: Enhance loyalty and maximize profitability in Parts and Service operations.
- **Vehicle Inventory Suite**: Utilize AI for smarter pricing and effective merchandising.
- **Intelligence Suite**: Leverage advanced analytics and machine learning for data-driven decision-making.

### Company Culture
With over 50 years of expertise, CDK Global thrives on empowerment and innovation. We foster a dynamic work environment where collaboration and continuous improvement are at the forefront. Our culture emphasizes building connections that facilitate automotive retail success. We value our employees and are committed to their growth, encouraging them to be adaptive and forward-thinking in a rapidly evolving industry.

### Careers at CDK
We are always on the lookout for talented and motivated individuals who are interested in making an impact in Automotive Retail. If you are ready to join a global leader in dealership software solutions and contribute to driving innovation in the industry, check out our job openings!

### Our Commitment to Customers
CDK Global is dedicated to resolving the pain points within the automotive retail sector. Our continuous efforts include insights through studies like the **Friction Points Study 2025**, aimed at identifying improvements and remaining challenges for our customers. 

### Contact Us
Interested in becoming part of the CDK Global family or learning more about our solutions? Reach out to us at:  
**Phone**: 833.620.2129  
**Email/Contact Form**: [Contact Us](link-to-contact-page)

Join us at CDK Global, where our mission is to create connections that empower your automotive retail success. We’re excited to help you reach your potential!

---

This brochure summarizes the key details about CDK Global's offerings, culture, and career opportunities. For further exploration, visit our [website](link-to-website).